In [1]:
import numpy as np
from PIL import Image
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
#对比展现原始图片和对抗样本图片
def show_images_diff(original_img,original_label,adversarial_img,adversarial_label):
    import matplotlib.pyplot as plt
    plt.figure()

    #归一化
    if original_img.any() > 1.0:
        original_img=original_img/255.0
    if adversarial_img.any() > 1.0:
        adversarial_img=adversarial_img/255.0

    plt.subplot(131)
    plt.title('Original')
    plt.imshow(original_img)
    plt.axis('off')

    plt.subplot(132)
    plt.title('Adversarial')
    plt.imshow(adversarial_img)
    plt.axis('off')

    plt.subplot(133)
    plt.title('Adversarial-Original')
    difference = adversarial_img - original_img
    #(-1,1)  -> (0,1)
    difference=difference / abs(difference).max()/2.0+0.5
    plt.imshow(difference,cmap=plt.cm.gray)
    plt.axis('off')
    plt.tight_layout()
    plt.show()

In [3]:
#加载解码的图像 这里是个大坑 tf提供的imagenet预训练好的模型pb文件中 包含针对图像的预处理环节 即解码jpg文件 这部分没有梯度
#需要直接处理解码后的数据
imagename="../picture/cropped_panda.jpg"
image=None
#with tf.gfile.Open(imagename, 'rb') as f:
    #image = np.array(Image.open(f).convert('RGB')).astype(np.float)
    #image=[image]
#    image_data = tf.gfile.FastGFile(imagename, 'rb').read()
image_data = tf.gfile.FastGFile(imagename, 'rb').read()

session=tf.Session()

def create_graph(dirname):
    with tf.gfile.FastGFile(dirname, 'rb') as f:
        graph_def = session.graph_def
        graph_def.ParseFromString(f.read())

        _ = tf.import_graph_def(graph_def, name='')

#从'http://download.tensorflow.org/models/image/imagenet/inception-2015-12-05.tgz'下载并解压到指定路径
create_graph("models/classify_image_graph_def.pb")

# 初始化参数  非常重要
session.run(tf.global_variables_initializer())
tensorlist=[n.name for n in session.graph_def.node]
#print(tensorlist)

#获取logits
#logits=session.graph.get_tensor_by_name('softmax:0')
#x = session.graph.get_tensor_by_name('DecodeJpeg/contents:0')

#label=session.run(logits,feed_dict={x:image_data})
#print(label)
#label=np.argmax(label)
#print(label)

softmax_tensor = session.graph.get_tensor_by_name('softmax:0')
predictions = session.run(softmax_tensor,
                           {'DecodeJpeg/contents:0': image_data})
predictions = np.squeeze(predictions)

print(np.argmax(predictions))

169


In [4]:
target = tf.placeholder(tf.float32, shape=[None, 1])
#交叉熵
# Use learning rate decay
global_step = tf.Variable(0, trainable=False)

adv = tf.Variable(initial_value=image, trainable=True,dtype=tf.float32)

cross_entropy = -tf.reduce_sum(target*tf.log(logits))
optimizer = tf.train.AdamOptimizer(0.01)
train_step=optimizer.minimize(loss=cross_entropy,var_list=[adv],global_step=global_step)

ValueError: initial_value must be specified.

In [ ]:
epochs=100
target=288

for epoch in range(epochs):
    
    train_step.run(feed_dict={x: x, target: [target]})
    
    label=session.run(logits,feed_dict={x:x})
    label=np.argmax(label)
    
    print("epoch={} loss={} label={}".format(epoch,loss,label))
    
    #如果定向攻击成功
    if label == target:
        break  
 